# Effect of Informal Care on Labor Supply: A Macroeconomic Perspective

## 2018.3.12 ICES, Hosei University

### Daisuke Moriwaki

Cyberagent, Inc., AI Lab        
Keio University, School of Medicine

<daisuke.moriwaki@gmail.com>

In [33]:
using QuantEcon
using Plots
using Interpolations
using Distributions

# Motivation
* Increase in the number of care recievers in Japan is inevitable. On the other hand, the human and financial resource that we can spend on care givers is getting scarse.
* It is easy to predict that "caregiver ratio" (the number of caregivers per one care receiver) explodes in the near future.

* The explosion of caregiver ratio cause several problems
    * insufficient care for elderly (insufficient availability of cheap nursing home, tokuyo)
    * Poverty due to care-leave for frail older family
    * Financial Problem for both national government and local administration
    * *Economy itself* due to less labor supply
* Such a wide range of problem should be tackled by macroeconomic model rather than causality studies.

# Literature

## labor supply and informal care

* moriwaki (2015, mimeo) finds negative effect from informal care only for male.

## care and macro economy


## heterogeneous agent and social security programs
* Li (EER, 2018)
* Kitao

# Setting

## Household

* you may consider either single households without gender and unknown re-production technique or many households with couples and kids where income, assets, health, and time endowment is shared. More realistic modelling is listed as future study
* 5 status space
    * age from 20 to 99 (5 year interval)
    * health status from 0 to 1. health with 0.1 or below becomes a care receiver.
    * wage rate is exogenous. Given wage rate, labor supply is endogeneous.
    * asset
    * *care* status; how many hours the household must spend for caring older frail family

## Preference
* simple utility function. setting is same as Yue (2018, EER)

In [34]:
function util(c::Real, l::Real,chi::Real = 0.4, sigma::Real = 0.28) # chi = 0.4
    ((c^(1-chi)*(l^chi))^(1-sigma))/(1 - sigma)
end

util (generic function with 3 methods)

## consumption
* consumption is deterimined by assets, income and saving
* there is a social assistence for the poor

## income
* income is determined by hourking hour in market and tax


## labor supply
* labor supply is endogeneous
* healthy household optimizes labor supply as well as consumption (saving) for given time endowment
* time endowment is varying across agent due to informal care
* From the first order condition $l = c(1 - \tau)w \frac{\chi}{1 - \chi}$

## Informal care
* Population is devided by health status. One is (potential) caregivers who are healthy enough to provide care to family, the other is care receivers who needs care from either/both formal or/and informal caregivers.
* Suppose most of the households prefer formal care, e.g. nursing home, to informal care at home. Then informal care occurs only when formal care provision is insufficient and this is exactly the case in Japan.
* Informal care is defined as unmet care demand from carereceivers. Informal care is distributed romdomly to caregivers according to caregivers' age.

## Government
* government provide insufficient formal care funded by social security premium contributed by working age, which is resemble to Japanese LTCI
* cost of formal care is determined by personal expenses of informal care worker

# Others
* wage ($w$) and capital cost $r$ is arbitrary determined by the nature (me).
* thus no equillibrium condition holds so far.
* $r = 0.01$, $\mathbb{E}(w) = 1$

## Quantitative Excercise
* The model is simulated with parameters set to values taken from existing literature.
* Unfortunately, I can only show the result of toy model so far.

## The toy model
* Households have only three period: young, middle, and old
* Young work only in the market and not give care.
* Middle work both in home and the market, but does not receive care.
* Old does not work.
* Wage is stochastic without persistent shocks.

In [35]:
r = 0.0
tau = 0.1 # income tax
beta = 0.9 # discount rate
rep_rate = 0.5 # replacement rate
NH_rate = 0.9 # government formal care provision rate. ratio to needed care
wage_care = 0.7 # care workers' wage rate. ratio to all industry

0.7

In [36]:
# calculating utility for old given asset and benefit
# need calculation for average_wage
function value_for_old(a::Real, average_wage::Real)
    b = rep_rate*average_wage # pension benefit
    return util((a*(1 + r) + b), 1)
end

value_for_old (generic function with 1 method)

In [65]:
# caluculating value for middle
# need calculaation for average wage
function value_for_middle(a_new::Real, l::Real, a::Real, w::Real, h_H::Real, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if c <= 0  || l <= 0 || 1 - h_H - l < 0
        v = -10000
    else
        v = util(c, l) + beta*value_for_old(a_new, w)
    end
    return v, a_new, l, a, w, h_H, y, c
end

value_for_middle (generic function with 3 methods)

In [121]:
# maximize value for middle given asset, wage and informal care
# row is new_asset x leisure
# column is wage x h_H x asset

min_asset = 0.001;
max_asset = 10;
min_wage = 0.001;
max_wage = 10;
min_h_H =0.0001;
max_h_H = 0.999;
num_grid = 9; #9
min_l = 0.0001;
max_l = 0.9999;
asset_grid = linspace(min_asset, max_asset, num_grid);
wage_grid = linspace(min_wage, max_wage, num_grid);
h_H_grid = linspace(min_h_H, max_h_H, num_grid);
l_grid = linspace(min_l, max_l, num_grid);





In [122]:
# calculate value for each input and return maximum value for each
# warinig: no interpolation embeded!
function generate_max_values_matrix(value_function::Function, asset_grid::LinSpace = asset_grid, l_grid::LinSpace = l_grid, wage_grid::LinSpace = wage_grid, 
        h_H_grid::LinSpace = h_H_grid, num_grid::Int = num_grid)
    # initializing matrices
    value_matrix = zeros(num_grid^2, num_grid^3);
    c_mat = zeros(num_grid^2, num_grid^3);
    l_mat = zeros(num_grid^2, num_grid^3);
    y_mat = zeros(num_grid^2, num_grid^3);
    a_new_mat = ones(num_grid^2, num_grid^3);
    w_mat = zeros(num_grid^2, num_grid^3);
    a_mat = zeros(num_grid^2, num_grid^3);
    h_H_mat = zeros(num_grid^2, num_grid^3);
    for i = 1:num_grid
        for j = 1:num_grid
            for k = 1:num_grid
                for m = 1:num_grid
                    for n = 1:num_grid
                        result = value_function(asset_grid[i], l_grid[j], asset_grid[k], wage_grid[m], h_H_grid[n])
                        value_matrix[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[1]
                        a_new_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[2]
                        l_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[3]
                        a_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[4]
                        w_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[5]
                        h_H_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[6]
                        y_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[7]
                        c_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2]  = result[8]
                    end
                end
            end
        end
    end
    # get max value for each asset, wage, h_H. maximizing with respect to new asset and leisure
    #max_value = maximum(value_matrix, 1)
    max_value, max_ind = findmax(value_matrix, 1)
    # reshape value matrix to asset_grid x (w x h_H)
    # this should be fix throughout the imulation
    max_values_reshaped = reshape(max_value, num_grid, num_grid^2);
    return max_values_reshaped, max_ind, a_new_mat, l_mat, a_mat, w_mat, h_H_mat, y_mat, c_mat
end


generate_max_values_matrix (generic function with 6 methods)

In [123]:
# get policy function for a_new, h_M (Labor supply), c, 
max_values_reshaped, max_ind, a_new_mat, l_mat, a_mat, w_mat, h_H_mat, y_mat, c_mat = generate_max_values_matrix(value_for_middle);
# policy function for new asset
a_new_pol_middle = a_new_mat[max_ind]
# policy function for labor supply
l_pol_middle = l_mat[max_ind]
h_M_pol_middle = 1 - l_pol_middle - reshape(h_H_mat[1, :], 1, num_grid^3)
c_pol_middle = c_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]


In [124]:
plot(reshape(c_pol_middle, 729, 1))
#plot(hcat(reshape(w_mat[1,:], 729,1), reshape(l_pol_middle, 729, 1)))

In [125]:
# need to be removed after get_value_for_middle_with_uncertainty is modified.

"get index for a value between grid point"
function get_float_index(x::Real, grid::LinSpace, num_grid::Int = num_grid)
# get floating index from asset grid for given scalar asset level
    if x < grid[1]
        return 1
    end
    if x> grid[num_grid]       
        return num_grid
    end
    for i = 1:num_grid
        if x >= grid[i]
            return i + (x - grid[i])/(grid[i+1] - grid[i])
        end
    end
end

get_float_index

In [126]:
max_values_matrix = generate_max_values_matrix(value_for_middle)[1]

9×81 Array{Float64,2}:
 0.116638  1.92083  2.59048  3.08603  …  2.05527  2.22362  2.37668  2.51777
 1.60466   2.56236  3.07077  3.49082     2.37512  2.51634  2.64783  2.77125
 2.90648   3.18155  3.52665  3.85716     2.64627  2.76982  2.88651  2.9973 
 3.44037   3.70373  3.93642  4.19872     2.88495  2.99587  3.10162  3.20281
 3.92066   4.10852  4.31564  4.52227     3.10006  3.20138  3.29864  3.39224
 4.31564   4.5035   4.67103  4.84212  …  3.29708  3.39081  3.48124  3.56865
 4.67103   4.8288   4.99633  5.14865     3.47968  3.56722  3.652    3.73424
 4.99633   5.14865  5.28905  5.43033     3.65044  3.73282  3.81283  3.89067
 5.28905   5.43033  5.57073  5.70148     3.81127  3.88925  3.96518  4.03922

In [127]:
############ need refactoring. interpolate accept "gridded" interpolation that replace get_float_index function!!!!!!!!!!!!!!!!!

# interpolation for a (new asset for young, asset in the last period for middle)
# get a = a_new and return value for middle with uncertainty
function get_value_for_middle_with_uncertainty(a_new::Real, asset_grid::LinSpace = asset_grid, max_values_matrix = max_values_matrix)
    v = zeros(num_grid^2, 1)
    a_new_ind = get_float_index(a_new, asset_grid) # get index of asset level in asset grid
    for i = 1:size(max_values_matrix, 2)
        itp = interpolate(max_values_matrix[:,i],BSpline(Linear()), OnGrid())
        v[i] = itp[a_new_ind]
    end
    return v
end

get_value_for_middle_with_uncertainty (generic function with 3 methods)

In [128]:
#plotly()
#plot(value_for_middle_mat[:, 30])
#plot(reshape(max_values, num_grid^3, 1))
#plot(max_values_reshaped)


## Distribution 
* assign probability distributions for wage and care need to each bin arbitrary
* wage follows lognormal, care need follow gamma distribution

In [129]:
# generate probability distribution for w and h_H
# wage is distributed according to Gauss


# calcurate probability at midpoint between each grid 
function get_Q(dist, grid::LinSpace, num_grid::Int=num_grid)
    Q = zeros(num_grid, 1)
    for i in 1:num_grid
        if i == 1
            Q[i] = cdf(dist, (grid[i] + grid[i + 1])/2)
        end
        if i == num_grid
            Q[i] = 1 - cdf(dist, (grid[i - 1] + grid[i])/2) 
        end
        if i > 1 && i < num_grid
            Q[i] = cdf(dist, (grid[i] + grid[i + 1])/2) - cdf(dist, (grid[i - 1] + grid[i])/2)
        end
    end
    return Q
end



get_Q (generic function with 2 methods)

In [130]:
w_dist = LogNormal(0.5, 0.5)
h_H_dist = Gamma(0.6, 0.1) # Gamma(0.5, 0.1)

# generate Q
Q_h_H = get_Q(h_H_dist, h_H_grid);
Q_w = get_Q(w_dist, wage_grid);
Q = kron(Q_w, Q_h_H);

In [131]:
# calculating value for young
"""function value_for_young(a_new::Real, l::Real, w::Real, Q::Array{Float64, 2}=Q, max_values_matrix::Array{Real, 2}=max_values_matrix, a::Real = 0, h_H::Real = 0, asset_grid::LinSpace = asset_grid, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if l < 0 || c < 0 || h_H + l > 1 || h_H + l < 0
        v = -10000
    else
        v = util(c, l) + beta*dot(Q, get_value_for_middle_with_uncertainty(a_new, asset_grid, max_values_matrix))
    end
return v, a_new, l, a, w, h_H, y, c
end

LoadError: syntax: incomplete: invalid string syntax

In [132]:
# calculating value for young
function value_for_young(a_new::Real, l::Real,  a::Real, w::Real, h_H::Real, v_new, Q = Q, asset_grid::LinSpace = asset_grid, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if l < 0 || c < 0 || h_H + l > 1 || h_H + l < 0
        v = -10000
    else
        v = util(c, l) + beta*dot(v_new, Q)
    end
return  v, a_new, l, a, w, h_H, y, c
end

value_for_young (generic function with 13 methods)

In [133]:
# caluculate value for young 


value_matrix = zeros(num_grid^2, num_grid^3);
c_mat = zeros(num_grid^2, num_grid^3);
l_mat = zeros(num_grid^2, num_grid^3);
y_mat = zeros(num_grid^2, num_grid^3);
a_new_mat = ones(num_grid^2, num_grid^3);
w_mat = zeros(num_grid^2, num_grid^3);
a_mat = zeros(num_grid^2, num_grid^3);
h_H_mat = zeros(num_grid^2, num_grid^3);
for i = 1:num_grid
    for j = 1:num_grid
        for k = 1:num_grid
            for m = 1:num_grid
                for n = 1:num_grid
                    v_new = get_value_for_middle_with_uncertainty(asset_grid[i])# value for middle with uncertainty
                    result = value_for_young(0, l_grid[j], 0, wage_grid[m], 0, v_new)
                    value_matrix[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[1]
                    a_new_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[2]
                    l_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[3]
                    a_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[4]
                    w_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[5]
                    h_H_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[6]
                    y_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[7]
                    c_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2]  = result[8]
                end
            end
        end
    end
end
# get max value for each asset, wage, h_H. maximizing with respect to new asset and leisure
max_value, max_ind = findmax(value_matrix, 1)
# reshape value matrix to asset_grid x (w x h_H)
# this should be fix throughout the imulation
max_values_reshaped = reshape(max_value, num_grid, num_grid^2);

In [134]:
# get policy function for a_new, h_M (Labor supply), c, 
# policy function for new asset
a_new_pol_young = a_new_mat[max_ind]
# policy function for labor supply
l_pol_young = l_mat[max_ind]
h_M_pol_young = 1 - l_pol_young - reshape(h_H_mat[1, :], 1, num_grid^3)
c_pol_young = c_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]

1×729 Array{Float64,2}:
 0.000562478  0.000562478  0.000562478  …  5.62478  5.62478  5.62478  5.62478

In [135]:
h_M_pol_young

1×729 Array{Float64,2}:
 0.624975  0.624975  0.624975  0.624975  …  0.624975  0.624975  0.624975

In [136]:
max_values_reshaped

9×81 Array{Float64,2}:
 4.69947  5.55774  5.87158  6.10379  …  6.46066  6.60835  6.74262  6.86638
 4.69947  5.55774  5.87158  6.10379     6.46066  6.60835  6.74262  6.86638
 4.69947  5.55774  5.87158  6.10379     6.46066  6.60835  6.74262  6.86638
 4.69947  5.55774  5.87158  6.10379     6.46066  6.60835  6.74262  6.86638
 4.69947  5.55774  5.87158  6.10379     6.46066  6.60835  6.74262  6.86638
 4.69947  5.55774  5.87158  6.10379  …  6.46066  6.60835  6.74262  6.86638
 4.69947  5.55774  5.87158  6.10379     6.46066  6.60835  6.74262  6.86638
 4.69947  5.55774  5.87158  6.10379     6.46066  6.60835  6.74262  6.86638
 4.69947  5.55774  5.87158  6.10379     6.46066  6.60835  6.74262  6.86638

In [137]:
?findmax


search: findmax findmax! findmin findmin!



```
findmax(itr) -> (x, index)
```

Returns the maximum element and its index. The collection must not be empty.

```jldoctest
julia> findmax([8,0.1,-9,pi])
(8.0,1)
```

```
findmax(A, region) -> (maxval, index)
```

For an array input, returns the value and index of the maximum over the given region.


In [138]:
# simulation 100 thousand H/H
# initializing matrices
num_period = 3;
num_hh = 100 # 100000
c_record = zeros(num_hh, num_period);
a_record = zeros(num_hh, num_period);
w_record = zeros(num_hh, num_period);
h_H_record = zeros(num_hh, num_period);
h_M_record = zeros(num_hh, num_period);



In [139]:
# generate wage, care needs streams for each household

w_record = rand(w_dist, num_hh, num_period);
w_record[:,3] = 0;
h_H_record = rand(h_H_dist, num_hh, num_period);
h_H_record[:, 1] = 0;
h_H_record[:, 3] = 0;


In [140]:
# return (c, a_new, h_M) for young hh
# prepare for interpolation
c_pol_young_reshape = reshape(c_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
a_new_pol_young_reshape = reshape(a_new_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
h_M_pol_young_reshape = reshape(h_M_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
c_itp = interpolate((asset_grid, wage_grid, h_H_grid), c_pol_young_reshape, Gridded(Linear()));
a_itp = interpolate((asset_grid, wage_grid, h_H_grid), a_new_pol_young_reshape, Gridded(Linear()));
h_M_itp = interpolate((asset_grid, wage_grid, h_H_grid), h_M_pol_young_reshape, Gridded(Linear()));

for n = 1:num_hh
    c_record[n,1] = c_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
    a_record[n,2] = a_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
    h_M_record[n,1] = h_M_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
end




In [141]:
# return (c, a_new, h_M) for middle

c_pol_middle_reshape = reshape(c_pol_middle, num_grid, num_grid, num_grid);
a_new_pol_middle_reshape = reshape(a_new_pol_middle, num_grid, num_grid, num_grid);
h_M_pol_middle_reshape = reshape(h_M_pol_middle, num_grid, num_grid, num_grid);
c_itp = interpolate((asset_grid, wage_grid, h_H_grid), c_pol_middle_reshape, Gridded(Linear()));
a_itp = interpolate((asset_grid, wage_grid, h_H_grid), a_new_pol_middle_reshape, Gridded(Linear()));
h_M_itp = interpolate((asset_grid, wage_grid, h_H_grid), h_M_pol_middle_reshape, Gridded(Linear()));


for n = 1:num_hh
    c_record[n,2] = c_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
    a_record[n,3] = a_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
    h_M_record[n,2] = h_M_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
end

for n = 1:num_hh
    c_record[n,3] = (1+r)*a_record[n, 3] + (w_record[n,1] + w_record[n, 2])*0.5*0.5 # replacement rate for pension is 0.5 
end

In [142]:
c_record

100×3 Array{Float64,2}:
 1.12458   1.12935   1.02904 
 0.509903  1.49266   1.02373 
 0.611696  1.10463   0.816661
 0.977086  0.811055  0.86824 
 1.38712   0.703495  0.931976
 0.9877    1.12362   1.06342 
 0.437306  0.671842  0.521557
 1.02263   0.702138  0.768116
 0.454662  0.726791  0.607053
 0.813289  0.934843  0.798459
 0.529742  0.852957  0.621131
 0.466412  0.474074  0.458392
 1.51029   0.84624   1.06663 
 ⋮                           
 0.585882  0.963079  0.693431
 0.852698  1.05663   0.881165
 0.683013  0.480039  0.550601
 1.56735   0.973542  1.18519 
 0.334337  0.531497  0.414955
 1.24223   0.714617  0.936663
 0.459548  0.653915  0.613735
 2.83301   0.627401  1.53943 
 0.519979  0.510865  0.513193
 0.508929  1.39962   0.900116
 1.05583   0.753099  0.814121
 1.36094   0.435956  0.810153

In [143]:
w_record

100×3 Array{Float64,2}:
 1.99933   2.11561   0.0
 0.90653   3.18839   0.0
 1.0875    2.17811   0.0
 1.73711   1.73339   0.0
 2.46609   1.25784   0.0
 1.75598   2.497     0.0
 0.777463  1.30494   0.0
 1.81807   1.25039   0.0
 0.808321  1.61591   0.0
 1.4459    1.74551   0.0
 0.941801  1.53965   0.0
 0.82921   1.00036   0.0
 2.68506   1.57853   0.0
 ⋮                      
 1.04161   1.72965   0.0
 1.51597   2.00711   0.0
 1.21429   0.98411   0.0
 2.7865    1.9525    0.0
 0.594401  1.06142   0.0
 2.20851   1.53505   0.0
 0.817007  1.63393   0.0
 5.03667   1.11704   0.0
 0.924443  1.12433   0.0
 0.904799  2.69566   0.0
 1.8771    1.37578   0.0
 2.41955   0.817067  0.0

In [144]:
h_M_record

100×3 Array{Float64,2}:
 0.624975  0.593338  0.0
 0.624975  0.520173  0.0
 0.624975  0.563669  0.0
 0.624975  0.520337  0.0
 0.624975  0.622312  0.0
 0.624975  0.500068  0.0
 0.624975  0.572879  0.0
 0.624975  0.624817  0.0
 0.624975  0.500462  0.0
 0.624975  0.595526  0.0
 0.624975  0.616162  0.0
 0.624975  0.527692  0.0
 0.624975  0.596239  0.0
 ⋮                      
 0.624975  0.619138  0.0
 0.624975  0.58521   0.0
 0.624975  0.543139  0.0
 0.624975  0.554317  0.0
 0.624975  0.557442  0.0
 0.624975  0.517862  0.0
 0.624975  0.445382  0.0
 0.624975  0.625076  0.0
 0.624975  0.505858  0.0
 0.624975  0.5769    0.0
 0.624975  0.608979  0.0
 0.624975  0.59424   0.0

In [145]:
h_H_record

100×3 Array{Float64,2}:
 0.0  0.0317525    0.0
 0.0  0.104802     0.0
 0.0  0.0613963    0.0
 0.0  0.104832     0.0
 0.0  0.00305806   0.0
 0.0  0.146636     0.0
 0.0  0.0524383    0.0
 0.0  0.000557281  0.0
 0.0  0.247517     0.0
 0.0  0.0296768    0.0
 0.0  0.00911499   0.0
 0.0  0.097625     0.0
 0.0  0.0290144    0.0
 ⋮                    
 0.0  0.00608089   0.0
 0.0  0.0399101    0.0
 0.0  0.0821915    0.0
 0.0  0.0708084    0.0
 0.0  0.0678937    0.0
 0.0  0.107356     0.0
 0.0  0.304876     0.0
 0.0  0.000309655  0.0
 0.0  0.119431     0.0
 0.0  0.0480747    0.0
 0.0  0.0163443    0.0
 0.0  0.0311444    0.0

In [146]:
sum(h_M_record)

119.35184309540679

In [147]:
sum(c_record)

297.9635068946603